In [7]:
#!jupyter nbextension enable --py gmaps

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [36]:
# Store csv created in part one into a DataFrame
city_weather = pd.read_csv("../output_data/cities.csv")
city_weather

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Novopokrovka,49.8425,36.5477,273.15,80,0,2.00,UA,1617142630
1,1,Torbay,47.6666,-52.7314,271.48,54,75,7.20,CA,1617142630
2,2,Saint-Pierre,-21.3393,55.4781,299.15,73,0,2.57,RE,1617142631
3,3,Lata,40.1629,-8.3327,292.15,45,0,2.06,PT,1617142631
4,4,Busselton,-33.6500,115.3333,287.04,96,51,2.19,AU,1617142631
...,...,...,...,...,...,...,...,...,...,...
554,554,Nouadhibou,20.9310,-17.0347,291.15,88,0,5.14,MR,1617142787
555,555,Martha Lake,47.8509,-122.2393,285.93,46,20,2.06,US,1617142787
556,556,Santa Clarita,34.3917,-118.5426,301.48,9,0,5.81,US,1617142406
557,557,Sabang,5.8933,95.3214,300.78,74,100,10.02,ID,1617142788


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [37]:
# Configure gmaps
gmaps.configure(api_key=g_key)

#store lat and lng into locations
locations = city_weather[['Lat', 'Lng']]

#get humidity
humidity = city_weather['Humidity']

In [38]:
# Heatmap of humidity
fig = gmaps.figure(center=(45.0, -5.0), zoom_level = 2)
map_intensity = np.max(humidity)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False,
                                max_intensity = map_intensity, point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [39]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_weather.loc[(city_weather['Wind Speed'] <= 10) & (city_weather['Cloudiness'] == 0) &
                                   (city_weather['Max Temp'] > 70) & (city_weather['Max Temp'] < 80)].dropna()

ideal_weather

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [46]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = pd.DataFrame(ideal_weather, columns=['City', 'Country', 'Lat', 'Lng'])
#create empty column for Hotel Name
hotel_df['Hotel Name'] = ''
hotel_df

,City,Country,Lat,Lng,Hotel Name


In [47]:
# Set parameters to search for a hotel
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# Iterate through 
for index, row in hotel_df.iterrows():
    
    # get lat, lng from df
    lat = row['Lat']
    lng = row['Lng']
    city_coordinates = f"{lat}, {lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    params = {'type' : 'hotel',
              'location' : city_coordinates,
             'keyword' : 'hotel',
             'radius' : 5000,
             'key' : g_key,}
    
    # make request and print url
    response = requests.get(base_url, params=params).json()
   
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
        print(f"Index {index} has a hotel: {response['results'][0]['name']}")
    except IndexError:
        print("Missing field/result...skipping.")
hotel_df

,City,Country,Lat,Lng,Hotel Name


In [48]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [49]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))